In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:

# Sigmoid activation function used in the neural network
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of the sigmoid function used in backpropagation
def sigmoid_derivative(x):
    return x * (1 - x)


In [3]:
# Cross-entropy loss function used to quantify the difference between prediction and observation
def cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-15  # to prevent log(0) cases
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # clip values to avoid NaNs in logarithms
    return -np.sum(y_true * np.log(y_pred)) / len(y_true)

# Flatten function to convert images into a 1D array and normalize pixel values
def flatten_image(image):
    return np.array(image).flatten() / 255.0

# Function to load images and labels from a folder
def load_images_from_folder(folder):
    images = [] #initializing empty list to store images
    labels = [] #initializing empty list to store labels

    class_folders = [class_folder for class_folder in os.listdir(folder) if os.path.isdir(os.path.join(folder, class_folder))]
   #it now contains the names of subdirectories within the specified folder
    


    for class_folder in class_folders: #loop iterates over each folder in the current class folder
        class_path = os.path.join(folder, class_folder) #creates a path to the current class folder

        for filename in os.listdir(class_path): #loop iterates over each image in the folder
            img_path = os.path.join(class_path, filename)
            if img_path.endswith(('.jpg', '.jpeg', '.png')):    #if the image is of this filetype 
                img = Image.open(img_path)                     #than we convert it into numpy array and append it to the images list
                images.append(np.array(img))  
                labels.append(class_folder)  #the name of the current class folder is appended to the labels list

    return np.array(images), np.array(labels)

In [4]:
# Class representing a simple feedforward neural network
class NeuralNetwork:

    # The constructor initializes the weights and biases with random values

    def __init__(self, input_size, hidden_size, output_size):
     #input size is the no. of input neurons
     #hidden size is the no. of hidden neurons
     #output size is the no. of output neurons                                                        

                                                            
        # Initialize weights and biases
        self.weights_input_hidden = np.random.rand(input_size, hidden_size) #randomly initializes weights connecting the input to the hidden layer
        self.weights_hidden_output = np.random.rand(hidden_size, output_size)#randomly initializes weights connecting the hidden to the output layer
        self.bias_hidden = np.zeros((1, hidden_size)) #bias value of hidden layer initialized to zeroes
        self.bias_output = np.zeros((1, output_size)) #bias value of output layer initialized to zeroes

    # Method to train the neural network using backpropagation
    def train(self, X, y, learning_rate, epochs): #x=input features, y=target labels, epochs=no. of iterations in train dataset
        
        for epoch in range(epochs):
            hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
            hidden_output = sigmoid(hidden_input)

            final_input = np.dot(hidden_output, self.weights_hidden_output) + self.bias_output
            final_output = sigmoid(final_input)

            error = y - final_output
            output_delta = error * sigmoid_derivative(final_output)

            hidden_layer_error = output_delta.dot(self.weights_hidden_output.T)
            hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_output)

            # Update weights and biases using gradient descent with cross-entropy loss
            self.weights_hidden_output += hidden_output.T.dot(output_delta) * learning_rate
            self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * learning_rate
            self.weights_input_hidden += X.T.dot(hidden_layer_delta) * learning_rate
            self.bias_hidden += np.sum(hidden_layer_delta, axis=0, keepdims=True) * learning_rate

            # Print cross-entropy loss for every 100 epochs
            if epoch % 100 == 0:
                ce_loss = cross_entropy_loss(y, final_output)
                print(f"Epoch {epoch}, Cross-Entropy Loss: {ce_loss}")

    # Method to make predictions using trained neural network
    def predict(self, X): #x is input data for which predictions are made

        hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        hidden_output = sigmoid(hidden_input)

        final_input = np.dot(hidden_output, self.weights_hidden_output) + self.bias_output
        final_output = sigmoid(final_input)

        return np.argmax(final_output, axis=1).reshape(-1)

In [5]:
# Main section
if __name__ == "__main__":
    train_folder_path = "/Users/rohanshenoy/Downloads/train"
    test_folder_path = "/Users/rohanshenoy/Downloads/test"

    # Load training images and labels
    train_images, train_labels = load_images_from_folder(train_folder_path)

    # Load testing images and labels
    test_images, test_labels = load_images_from_folder(test_folder_path)

    # Encode labels using scikit-learn LabelEncoder
    label_encoder = LabelEncoder()
    encoded_train_labels = label_encoder.fit_transform(train_labels)
    encoded_test_labels = label_encoder.transform(test_labels)

    # Flatten the images and normalize them
    train_images = np.array([flatten_image(img) for img in train_images])
    test_images = np.array([flatten_image(img) for img in test_images])

    # Ensure labels have the correct shape
    encoded_train_labels = encoded_train_labels.reshape(-1, 1)  # Reshape to (num_samples, 1)
    encoded_test_labels = encoded_test_labels.flatten()

    # Split the data into training and testing sets
    train_images, val_images, train_labels, val_labels = train_test_split(train_images, encoded_train_labels, test_size=0.2, random_state=42)

    # Initialize and train the neural network
    neural_net = NeuralNetwork(input_size=len(flatten_image(train_images[0])), hidden_size=4, output_size=len(np.unique(encoded_train_labels)))
    neural_net.train(train_images, train_labels, learning_rate=0.1, epochs=1000)

    # Make predictions on the validation set
    val_predictions = neural_net.predict(val_images)
    val_predicted_labels = np.round(val_predictions)

    print("Unique values in val_labels:", np.unique(val_labels))
    print("Unique values in val_predicted_labels:", np.unique(val_predicted_labels))

    # Calculate accuracy on the validation set
    val_accuracy = accuracy_score(val_labels, val_predicted_labels)
    print(f"Validation Accuracy: {val_accuracy}")

    # Make predictions on the test set
    test_predictions = neural_net.predict(test_images)
    test_predicted_labels = np.round(test_predictions)

    # Calculate accuracy on the test set
    test_accuracy = accuracy_score(encoded_test_labels, test_predicted_labels)
    print(f"Test Accuracy: {test_accuracy}")

Epoch 0, Cross-Entropy Loss: 5.3392975276460515
Epoch 100, Cross-Entropy Loss: 4.5123280112413516e-14
Epoch 200, Cross-Entropy Loss: 4.5123280112413516e-14
Epoch 300, Cross-Entropy Loss: 4.5123280112413516e-14
Epoch 400, Cross-Entropy Loss: 4.5123280112413516e-14
Epoch 500, Cross-Entropy Loss: 4.5123280112413516e-14
Epoch 600, Cross-Entropy Loss: 4.5123280112413516e-14
Epoch 700, Cross-Entropy Loss: 4.5123280112413516e-14
Epoch 800, Cross-Entropy Loss: 4.5123280112413516e-14
Epoch 900, Cross-Entropy Loss: 4.5123280112413516e-14
Unique values in val_labels: [0 1 2 3 4 5 6 7 8 9]
Unique values in val_predicted_labels: [0]
Validation Accuracy: 0.10375
Test Accuracy: 0.1
